In [ ]:
queries = ["机票","值机","摆渡","餐饮","座位","空姐","安检","客服","晚点","贵宾"]
textname = "text.csv"
tofile = "result.csv"

In [1]:
!pip install rank_bm25
!pip install snownlp
!pip install cocoNLP

     |████████████████████████████████| 37.6MB 121kB/s 
  Created wheel for snownlp: filename=snownlp-0.12.3-cp36-none-any.whl size=37760957 sha256=7288519897000d1702955dc64af4ebd2051437e7909492379c6694f151c8f7ca
  Stored in directory: /root/.cache/pip/wheels/f3/81/25/7c197493bd7daf177016f1a951c5c3a53b1c7e9339fd11ec8f
Successfully built snownlp
     |████████████████████████████████| 92kB 3.6MB/s 
  Using cached https://files.pythonhosted.org/packages/8f/99/13078d71bc9f77705a29f932359046abac3001335ea1d21e91120b200b21/pyhanlp-0.1.66.tar.gz
  Using cached https://files.pythonhosted.org/packages/b2/7e/7ff0d15fafd109d40ffa50f589aa793cdee6fd00e667c5efed02bd6c35dd/phone-0.4.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/c1/ac/57ea656c1ff9a8b5dc9729d9b75f4d568c3598d4e4a7df31c1791ed9a095/phonenumbers-8.12.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/bc/ebc1afb3c54377e128a01024c006f983d03ee124bc52392b78ba98c421b8/arrow-0.17.0-py2.py3-none-an

In [28]:
from collections import Counter
class prores:
  resstr = "";
  resl = [];
  
def process(rank_score):
  res = prores();
  resstr = ""
  resl = []
  for t in rank_score:
    res.resstr += t[1] + " "
  res.resstr = res.resstr[:-1]
    
  res.resl = Counter(res.resstr.split(" ")).most_common(10)
  return res;

In [29]:
from rank_bm25 import BM25Plus
import pandas as pd
from snownlp import SnowNLP
from cocoNLP.config.phrase import rake
import cocoNLP
import pandas as pd
#coding:utf-8
text = pd.read_excel(textname,names=["text"])
# text = pd.read_excel("text.csv",names=["text"])
# print(text["text"])
corpus = text["text"].tolist()
stop = [line.strip() for line in open('stop_words.txt', encoding='utf-8').readlines() ]

def remStp(words):
  ans = ""
  for word in words:
    if word not in stop: ans += word
  return ans

def fetch(query):
  col1_ori = []
  col2_tok = []
  col3_freq = []
  col0_q = []

  r = rake.Rake()
  tokenized_corpus = [remStp(SnowNLP(doc).words) for doc in corpus]
  print(tokenized_corpus)
  bm25 = BM25Plus(tokenized_corpus)
  # query = "空乘人员"
  tokenized_query = SnowNLP(query).words
  res = bm25.get_top_n(tokenized_query, tokenized_corpus, n=10)
  # print(res)
  for sen in res:
      r.extract_keywords_from_sentences([sen],2, 4)

      ranked_words = r.get_ranked_phrases()

      ranked_words_score = r.get_ranked_phrases_with_scores()

      
      theprocess = process(ranked_words_score)
      col0_q += [query]
      col1_ori += [sen]
      col2_tok += [theprocess.resstr]
      col3_freq += [theprocess.resl]
      # print(sen+"\t"+str(ranked_words_score))
  df = pd.DataFrame({"query":col0_q,"text":col1_ori,"token":col2_tok,"frequency":col3_freq })
  return df

In [30]:
# queries = ["机票","值机","摆渡","餐饮","座位","空姐","安检","客服","晚点","贵宾"]
# query = "摆渡"
df = fetch(queries[0]);
for i in range(1,len(queries)):
  df = df.append([fetch(queries[i])],ignore_index=True)
# df.to_csv("result4.csv")
df.to_csv(tofile)

['问题延误小时郁闷', '升舱餐食？！', '安全员起飞过程中睡着，CC全程问候', '趟飞机飞行过程颠簸', '手机落飞机商务舱！！！', '更换机型及时通知擅自更改已经选定座位通知高端经济舱赚钱一直南方航空表现比较满意今天太令人失望', '空培训佳礼貌问候餐食口味差温度不够热', '飞机摆渡', '视听节目广告时间过长广告结束继续播放', '公务舱座椅太难受如坐针毡', '饭夹生鸡肉新鲜', '首都机场地勤指挥真好好关键好好践行群众路线，T3跑道落地开T2航站楼东侧最北端掉头T2航站楼东侧最南端绕T2航站楼西侧平台最远端摆渡车回航站楼落地抵达航站楼超过40分钟', '降落期间操作太凶顿操作猛如虎离心力太', '定票钱时已经选好座位飞机起飞前天座位不见重新选后座位甚短信提示不好导致心情变不好', '我用公务卡买机票南航app里没法进行改签希望方面进行改进。。', '设备太差', '合并航班调整已经选定的舱位座位请及时予以客人告知联系直接系统调整', '这班航班服务差', '冷天下飞机是廊桥太受罪', '餐饮品种少主食种选择', '昆明机场南航贵宾休息室太人遗憾食物到点分钟迟疑收乘客走大量餐盘与人过问候机室小可怜空气中弥漫难闻气味两舱客人金银卡共用休息室登机口太远知道安排安排一个供客人随时食用米线餐难事', '机上无餐饮', '服务态度不好不够热情', '忘记配餐', '电视无法使用', '座椅娱乐设备故障', '机餐', '机上湿巾太差', '网上值机后希望自助打印登机牌', '航班座位??电视??', '乘务员讲，“麻烦前面多余腰枕一个”，说经济舱腰枕提供经济舱航班坐空位我才讲多余回感觉真不爽努力赚钱买头等舱', '登机口号改212登机结束APP7，改么电子登机牌只关注这破APP，南航短信提醒误机只能说APP太烂', '公务舱应待遇相同到达北京），厚薄', '飞机座椅舒服餐饮合口味满意', '早晚吃包子？！！！！！！！！！！！！！！！！！！', '位置空间不足', '买明珠经济舱标榜座椅舒适实际差距每个座位固定放置毛毯腰枕更是座椅空间弄狭窄空间普通经济舱差餐饮配发差明明位置前三排配餐种类不足情况乘务员未经询问直接发餐主食内容作介绍解释感觉机器人总体体验不好', '值机时工作人员强烈等待值机客人扫二维码说扫二维码取票明明选座位网上值机第二工作人员听客人说只说扫二

In [16]:
# from collections import Counter
# # detail = Counter(a[0][1].split(" "))
# # print(detail.most_common(3))
# rr = process(a)
# print(rr.resstr)
# print(rr.resl)
print(remStp(SnowNLP("/5星服务态度好;响应及时;职业形象佳;客舱广播清晰/4星餐食口味好/5星客舱环境舒适;娱乐节目丰富/5星服务态度好;职业形象佳;行李交付较快").words))

['服务', '态度', '好', '响应', '及时', '职业', '形象', '佳', '客舱', '广播', '清晰', '餐食', '口味', '好', '客舱', '环境', '舒适', '娱乐', '节目', '丰富', '服务', '态度', '好', '职业', '形象', '佳', '行李', '交付']


In [ ]:
餐饮，座位，空姐，安检，客服，晚点